The current script is used to import the Excel workbook which contains data from the research done by Gert-Jan Steenveld. The data will be selected based on the time period under review, namely the first of June 2017 until the 31st of May 2018. The dataset will then be stored to be used in the later validation of models produced in this project 

Install required packages if not present, if so uncomment next cell, and load libraries

In [14]:
#import sys
#!{sys.executable} -m pip install openpyxl
#!{sys.executable} -m pip install pandas
#!{sys.executable} -m pip install datetime

In [24]:
import openpyxl
import pandas as pd
import datetime 

Read the sheets of the validation dataset into a dictionary, create an empty dictionary to store the transformed data. Set the start and end time of the project for selection of relevant measurements

In [25]:
all_data = pd.read_excel("val_dat.xlsx",sheet_name=None)
year_data  = {}
start_time = datetime.datetime.strptime("01/06/2017 00:00","%d/%m/%Y %H:%M")
end_time = datetime.datetime.strptime("31/05/2018 23:59","%d/%m/%Y %H:%M")

Create a function to remove one erronous float value that prevents the date time strings being converted into datetime

In [27]:
def rem_float(x):
    try:
        float(x)
    except ValueError:
        return True
    return False

Loop over the keys of the dictionary which hold the dataframes containing the measurements for the related stations. For each dataframe, remove the possible float value in the UTC column holding strings containing measurement date and time. Then convert the column to datetime format and select the relevant time period. Store the converted dataframe in a new dictionary.

In [29]:
for key in all_data:
    df = all_data[key]
    searchfloat = df["UTC"].apply(rem_float)
    df = df[searchfloat]
    df["UTC"] = pd.to_datetime(df["UTC"],format=" %d/%m/%Y %H:%M")
    df = df[(df["UTC"] >= start_time) & (df["UTC"] <= end_time)]
    year_data[key] = df

<ipython-input-29-7768dde9c7a2>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["UTC"] = pd.to_datetime(df["UTC"],format=" %d/%m/%Y %H:%M")


Write the dataframes back into an excel workbook to use for validation later

In [30]:
with pd.ExcelWriter('validation.xlsx') as writer:  
    for entry in year_data:
        df1 = year_data[entry]
        df1.to_excel(writer, sheet_name=entry)

To conclude, the main difficulty was managing to load, use and write a XSLX workbook which has multiple sheets in pandas. Furthermore, while most of the UTC values were strings, some sheets contained a random number which prevented use of the efficient pd.to_datetime as the float did not have the correct format. Finding and removing this float value was challenging but a stark improvement from looping over all the seperate values and checking whether they were the correct format. 

This script was used to produce a dataset that was meant to validate the outcomes of the models. However, as the models had such low accuracy, the produced file was never used. Apart from that, the conversion and the shrinking of the data set by selecting a fixed time interval improved processing time.